# File used to map the density and number of flowers in a patch to the distance between neareast-neighbouring flowers

### Step 1: Import all packages and define the current working directory

In [1]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd

import os
current_working_directory = os.getcwd()
arrays_dir = current_working_directory + '\\Arrays'

### Step 2: Define the functions for environment generation

In [2]:
def convert_polar_to_cartesian (polar_vector) : 
	cartesian_vector = [polar_vector[0] * np.cos(polar_vector[1]) , polar_vector[0] * np.sin(polar_vector[1])]
	return(cartesian_vector)

def distance(point1,point2) : 
	return(np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2))


def generate_cartesian_coordinates_for_flower(min_distance,max_distance) :  # can be used for patch centers

  distance = np.random.uniform(low=min_distance,high=max_distance)
  azimuth = np.random.uniform(low=0,high=2*np.pi)
  return(convert_polar_to_cartesian((distance,azimuth)))


def check_if_patch_is_sufficiently_far(coordinates_of_current_patch, minimal_distance_between_patches ,coordinates_of_other_patches) : # can be used for path centers

  number_of_patches_to_check, _ = np.shape(coordinates_of_other_patches)
  distances_with_other_patches = [distance(coordinates_of_current_patch,coordinates_of_other_patches[flower]) for flower in range(number_of_patches_to_check)]

  return (np.min(distances_with_other_patches)>=minimal_distance_between_patches)

In [3]:
def generate_array_procedurally(number_of_flowers,number_of_patches,density_of_patches,foraging_range=1000,perception_range=5) : 

  if density_of_patches < 0 : 
    raise ValueError("Unsupported density of patches. Values must be positive.")
  if number_of_patches > number_of_flowers : 
    raise ValueError("You must have at least one flower per patch.")
  if number_of_flowers == 0 or number_of_patches == 0 :
    raise ValueError("You must at least have one flower and one patch.")
  if (4*perception_range >= foraging_range) :
    raise ValueError("The environment size must be at least superior to 4 times the perception range.")

  MARGIN_OF_ENVIRONMENT = 10
  MINIMAL_DISTANCE_PATCH_PATCH = 2*perception_range
  MAX_TRIALS_TO_PLACE_CENTER = 200
  INCREASE_ENVIRONMENT_SIZE = foraging_range/20


  # Special case if density_of_patches == 0: uniform environment. We create as much patches as there are flowers in the environment
  if density_of_patches == 0:
    number_of_patches = number_of_flowers

  # Draw the coordinates of the centers of the patches and check if they respect the minimal distance between patches (the nest is considered as a patch center here)   

  patch_centers_coordinates = [np.array([0,0])]

  while(len(patch_centers_coordinates)!=number_of_patches+1) :
    coordinates_of_current_patch = np.random.uniform(low = -foraging_range+MARGIN_OF_ENVIRONMENT, high = foraging_range-MARGIN_OF_ENVIRONMENT, size=2)   
    patch_center_far_enough = check_if_patch_is_sufficiently_far(coordinates_of_current_patch, MINIMAL_DISTANCE_PATCH_PATCH, patch_centers_coordinates) 
    trial = 0

    while (not patch_center_far_enough) and trial<MAX_TRIALS_TO_PLACE_CENTER : 
      coordinates_of_current_patch = np.random.uniform(low = -foraging_range+MARGIN_OF_ENVIRONMENT, high = foraging_range-MARGIN_OF_ENVIRONMENT, size=2)   
      patch_center_far_enough = check_if_patch_is_sufficiently_far(coordinates_of_current_patch, MINIMAL_DISTANCE_PATCH_PATCH, patch_centers_coordinates)
      trial += 1

    if patch_center_far_enough : 
      patch_centers_coordinates.append(coordinates_of_current_patch)

    else : 
      foraging_range += INCREASE_ENVIRONMENT_SIZE

  # If density_of_patches == 0: the patch centers will be the flowers
  if density_of_patches == 0:
    patch_centers_coordinates = np.array(patch_centers_coordinates)
  else : 

    # For each flower, decide the patch to which it will belong (patch 0 is the nest so there will be no flower assigned to it)
    flowers_ID_coordinates_patch = [np.array([0,0,0,0])] # Contains the future output (flower ID, x coordinate, y coordinate, patch ID). The nest is considered a flower (in patch 0)
    flowers_belonging_to_patch = np.random.randint(low = 1, high = (number_of_patches+1), size = number_of_flowers)
    number_of_flower_per_patch = np.bincount(flowers_belonging_to_patch,minlength=number_of_patches+1)

    flower_ID = 1

    for patch in range (number_of_patches+1) :
      
      radius_of_patch = np.sqrt(number_of_flower_per_patch[patch]/(density_of_patches*np.pi))

      for flower in range(number_of_flower_per_patch[patch]) : 

        x,y = generate_cartesian_coordinates_for_flower(0,radius_of_patch)
        coordinates_of_flower = patch_centers_coordinates[patch]+np.array([x,y])
        flowers_ID_coordinates_patch.append(np.concatenate(([flower_ID],coordinates_of_flower,[patch])))
        flower_ID += 1


  return(np.array(flowers_ID_coordinates_patch)[:,1:3])

### Step 3: Compute the mean nearest-neighbour distance in a given array

In [4]:
def get_matrix_of_distances_between_flowers(matrix_of_coordinates) : 
	matrix_of_pairwise_distances = euclidean_distances (matrix_of_coordinates,matrix_of_coordinates)
	return(matrix_of_pairwise_distances)

def mean_nn_dist(number_of_flowers,number_of_patches,density_of_patches, nrep):
    l=[]
    for rep in range (nrep):
        mat_coord = generate_array_procedurally(number_of_flowers,number_of_patches,density_of_patches)
        mat_dist = get_matrix_of_distances_between_flowers(mat_coord)
        for i in range (1,len(mat_dist)) : #exclude distance from nest
            l.append(np.min(mat_dist[i, mat_dist[i]>0]))
    return(np.mean(l))

### Step 4: Generate the equivalence dataframes

In [6]:
fl = []
pat = []
dens = []
nn_dist = []


for number_of_flowers in [25] : 
    for number_of_patches in [5] :
        for density_of_patches in [0.1,0.01,0.001,0.0001,0.00001,0.000001, 0.0000001] : 
            fl.append(number_of_flowers)
            pat.append(number_of_patches)
            dens.append(density_of_patches)
            nn_dist.append(mean_nn_dist(number_of_flowers,number_of_patches,density_of_patches,10000))

res = pd.DataFrame({"number_of_flowers":fl,"number_of_patches":pat,"density_of_patches":dens, "average NN distance":nn_dist})  
res.to_csv(arrays_dir+"\\equivalence_NN_5patches.csv") 

In [9]:
fl = []
pat = []
dens = []
nn_dist = []


for number_of_flowers in [5,10,20,25,40] : 
    for number_of_patches in [1] :
        for density_of_patches in [0.1,0.01,0.001,0.0001,0.00001, 0.000001]  : 
            fl.append(number_of_flowers)
            pat.append(number_of_patches)
            dens.append(density_of_patches)
            nn_dist.append(mean_nn_dist(number_of_flowers,number_of_patches,density_of_patches,10000))

res = pd.DataFrame({"number_of_flowers":fl,"number_of_patches":pat,"density_of_patches":dens, "average NN distance":nn_dist}) 
res.to_csv(arrays_dir+"\\equivalence_NN_1patch.csv")